# Wildfire Prediction - Data Prep

In [1095]:
import json
import pandas as pd
import requests

from datetime import datetime
from io import StringIO

## Get NOAA weather data

In [1096]:
# NOAA data

# from https://www.ncei.noaa.gov/pub/data/cirs/climdiv/
# README file: https://www.ncei.noaa.gov/pub/data/cirs/climdiv/county-readme.txt
# in this file CA state code is 04 (not 06)

noaa_pcp_url = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-pcpncy-v1.0.0-20240705"
noaa_tmax_url = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-tmaxcy-v1.0.0-20240705"
noaa_tmin_url = "https://www.ncei.noaa.gov/pub/data/cirs/climdiv/climdiv-tmincy-v1.0.0-20240705"

In [1097]:
def get_noaa_data_df(target_url):
    noaa_data = requests.get(target_url)    
    noaa_month_colnames= ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
    noaa_df = pd.read_csv(StringIO(noaa_data.text), lineterminator="\n", sep=r"\s+", header=None, names=["ID", *noaa_month_colnames], index_col=False, dtype="string")
    return noaa_df

In [1098]:
pcp_data_df = get_noaa_data_df(noaa_pcp_url)
pcp_data_df.head(2)

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,01001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66
1,01001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94


In [1099]:
tmax_data_df = get_noaa_data_df(noaa_tmax_url)
tmax_data_df.head(2)

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,01001271895,53.70,48.70,67.60,76.40,81.90,89.20,91.10,90.40,90.90,76.00,66.60,58.00
1,01001271896,54.20,60.80,65.30,81.60,88.50,88.20,92.00,94.50,90.80,77.20,69.90,58.70


In [1100]:
tmin_data_df = get_noaa_data_df(noaa_tmin_url)
tmin_data_df.head(2)

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,01001281895,34.20,27.70,43.40,51.80,59.30,67.40,69.70,70.30,67.10,46.90,42.10,32.50
1,01001281896,34.40,37.20,42.60,57.00,65.00,67.90,71.40,71.70,65.00,52.20,46.10,35.90


In [1101]:
# Concatenate NOAA dfs together
noaa_full_df = pd.concat([pcp_data_df, tmax_data_df, tmin_data_df], axis=0)

## Parse NOAA weather data

In [1102]:
# Extract NOAA ID
def extract_noaa_id(df):
    df["STATE_CODE"] = df["ID"].str[:2]
    df["FIPS_CODE"] = df["ID"].str[2:5]
    df["ELEMENT_CODE"] = df["ID"].str[5:7]
    df["YEAR"] = df["ID"].str[7:]
    return df

extract_noaa_id(noaa_full_df)
noaa_full_df

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR
0,01001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66,01,001,01,1895
1,01001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94,01,001,01,1896
2,01001011897,3.27,6.63,10.94,4.35,0.81,1.57,3.96,5.02,0.87,0.75,1.84,4.38,01,001,01,1897
3,01001011898,2.33,2.07,2.60,4.56,0.54,3.13,5.80,6.02,1.51,3.21,6.66,3.91,01,001,01,1898
4,01001011899,5.80,6.94,3.35,2.22,2.93,2.31,6.80,2.90,0.63,3.02,1.98,5.25,01,001,01,1899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406905,50290282020,-22.80,-16.70,-1.60,18.40,36.30,45.90,47.10,44.80,34.90,18.30,-0.90,-3.20,50,290,28,2020
406906,50290282021,-3.30,-17.20,-6.20,11.00,33.30,46.20,49.20,43.90,31.10,20.50,-7.50,-7.60,50,290,28,2021
406907,50290282022,-12.50,-9.80,2.20,12.80,33.00,45.10,47.40,43.50,35.80,19.60,3.00,-8.50,50,290,28,2022
406908,50290282023,-4.40,-9.40,-1.70,2.10,34.60,45.10,50.80,48.00,33.00,17.20,8.80,-10.80,50,290,28,2023


In [1103]:
# Map element codes to descriptive abbreviation

element_code_map = {
# element codes from docs
    "01": "pcp", # Precipitation
    "02": "tavg", # Average Temperature
    "25": "Heating Degree Days",
    "26": "Cooling Degree Days",
    "27": "tmax", # Maximum Temperature
    "28": "tmin" # Minimum Temperature
}

noaa_full_df["NOAA_ELEMENT"] = noaa_full_df["ELEMENT_CODE"].map(element_code_map).fillna(noaa_full_df["ELEMENT_CODE"])
noaa_full_df

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR,NOAA_ELEMENT
0,01001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66,01,001,01,1895,pcp
1,01001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94,01,001,01,1896,pcp
2,01001011897,3.27,6.63,10.94,4.35,0.81,1.57,3.96,5.02,0.87,0.75,1.84,4.38,01,001,01,1897,pcp
3,01001011898,2.33,2.07,2.60,4.56,0.54,3.13,5.80,6.02,1.51,3.21,6.66,3.91,01,001,01,1898,pcp
4,01001011899,5.80,6.94,3.35,2.22,2.93,2.31,6.80,2.90,0.63,3.02,1.98,5.25,01,001,01,1899,pcp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406905,50290282020,-22.80,-16.70,-1.60,18.40,36.30,45.90,47.10,44.80,34.90,18.30,-0.90,-3.20,50,290,28,2020,tmin
406906,50290282021,-3.30,-17.20,-6.20,11.00,33.30,46.20,49.20,43.90,31.10,20.50,-7.50,-7.60,50,290,28,2021,tmin
406907,50290282022,-12.50,-9.80,2.20,12.80,33.00,45.10,47.40,43.50,35.80,19.60,3.00,-8.50,50,290,28,2022,tmin
406908,50290282023,-4.40,-9.40,-1.70,2.10,34.60,45.10,50.80,48.00,33.00,17.20,8.80,-10.80,50,290,28,2023,tmin


## Create cleaned subset with just CA counties

In [1104]:
noaa_ca_df = noaa_full_df[noaa_full_df["STATE_CODE"] == "04"]
noaa_ca_df

,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR,NOAA_ELEMENT
20410,04001011895,8.43,2.09,1.97,1.75,1.20,0.00,0.05,0.01,0.50,0.57,1.57,1.47,04,001,01,1895,pcp
20411,04001011896,8.47,0.23,2.71,4.13,0.72,0.00,0.02,0.27,0.47,1.56,4.12,3.30,04,001,01,1896,pcp
20412,04001011897,2.66,4.30,4.11,0.56,0.14,0.21,0.00,0.02,0.15,1.82,0.97,1.54,04,001,01,1897,pcp
20413,04001011898,1.14,2.49,0.48,0.21,1.35,0.18,0.00,0.02,0.86,0.95,0.65,1.32,04,001,01,1898,pcp
20414,04001011899,4.52,0.25,7.19,0.55,0.86,0.22,0.00,0.04,0.03,4.20,3.69,2.65,04,001,01,1899,pcp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27945,04115282020,38.90,39.70,40.60,47.10,53.30,58.60,62.10,65.00,61.10,53.30,40.40,37.40,04,115,28,2020,tmin
27946,04115282021,38.50,39.40,39.50,46.70,53.40,61.40,66.40,63.40,59.00,48.50,45.70,37.70,04,115,28,2021,tmin
27947,04115282022,37.10,37.80,43.20,44.70,50.50,59.00,62.40,64.10,61.40,51.80,38.00,36.90,04,115,28,2022,tmin
27948,04115282023,36.80,35.50,37.80,44.10,52.00,56.20,63.70,63.10,56.10,50.60,42.70,41.10,04,115,28,2023,tmin


## Get county FIPS codes

In [1105]:
# FIPS Codes

# from https://www.census.gov/library/reference/code-lists/ansi.html#cou
fips_url = "https://www2.census.gov/geo/docs/reference/codes2020/cou/st06_ca_cou2020.txt"

fips_data = requests.get(fips_url)
fips_df = pd.read_csv(StringIO(fips_data.text), lineterminator="\n", sep="|", dtype="string")
fips_df.head(2)

,STATE,STATEFP,COUNTYFP,COUNTYNS,COUNTYNAME,CLASSFP,FUNCSTAT
0,CA,06,001,01675839,Alameda County,H1,A
1,CA,06,003,01675840,Alpine County,H1,A


In [1106]:
# Join to FIPS df
noaa_ca_counties_df = pd.merge(fips_df[["COUNTYFP", "COUNTYNAME"]], noaa_ca_df, left_on="COUNTYFP", right_on="FIPS_CODE")
noaa_ca_counties_df["COUNTYNAME"] = noaa_ca_counties_df["COUNTYNAME"].str.split().apply(lambda row: ' '.join(row[:-1]))
noaa_ca_counties_df.head(2)

,COUNTYFP,COUNTYNAME,ID,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,STATE_CODE,FIPS_CODE,ELEMENT_CODE,YEAR,NOAA_ELEMENT
0,001,Alameda,04001011895,8.43,2.09,1.97,1.75,1.20,0.00,0.05,0.01,0.50,0.57,1.57,1.47,04,001,01,1895,pcp
1,001,Alameda,04001011896,8.47,0.23,2.71,4.13,0.72,0.00,0.02,0.27,0.47,1.56,4.12,3.30,04,001,01,1896,pcp


In [1107]:
# Remove unnecessary encoding columns
noaa_ca_counties_df = noaa_ca_counties_df[["COUNTYNAME", "YEAR", 'NOAA_ELEMENT', 'JAN', 'FEB', 'MAR', 'APR', 'MAY',
       'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'ID']]
noaa_ca_counties_df.head(2)

,COUNTYNAME,YEAR,NOAA_ELEMENT,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ID
0,Alameda,1895,pcp,8.43,2.09,1.97,1.75,1.20,0.00,0.05,0.01,0.50,0.57,1.57,1.47,04001011895
1,Alameda,1896,pcp,8.47,0.23,2.71,4.13,0.72,0.00,0.02,0.27,0.47,1.56,4.12,3.30,04001011896


In [1108]:
noaa_long = noaa_ca_counties_df.melt(id_vars=['COUNTYNAME', 'ID', 'YEAR', 'NOAA_ELEMENT'], var_name='MONTH', value_name='NOAA_ELEMENT_VAL')
noaa_long['NOAA_ELEMENT_VAL'] = noaa_long['NOAA_ELEMENT_VAL'].astype(float)
noaa_long['MONTH_NUM'] = noaa_long['MONTH'].apply(
    lambda month_str: datetime.strptime(month_str, '%b').month)
noaa_long.head(2)

,COUNTYNAME,ID,YEAR,NOAA_ELEMENT,MONTH,NOAA_ELEMENT_VAL,MONTH_NUM
0,Alameda,04001011895,1895,pcp,JAN,8.43,1
1,Alameda,04001011896,1896,pcp,JAN,8.47,1


In [1109]:
noaa_pivot_df = noaa_long.pivot_table(
    index=['COUNTYNAME', 'YEAR', 'MONTH_NUM', 'MONTH'],
    columns='NOAA_ELEMENT',
    values='NOAA_ELEMENT_VAL'
).sort_values(by=["COUNTYNAME", "YEAR", "MONTH_NUM"]).reset_index()

noaa_pivot_df["YEAR"] = noaa_pivot_df["YEAR"].astype(float)
noaa_pivot_df["MONTH_NUM"] = noaa_pivot_df["MONTH_NUM"].astype(float)

noaa_pivot_df

NOAA_ELEMENT,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin
0,Alameda,1895.0,1.0,JAN,8.43,51.5,38.8
1,Alameda,1895.0,2.0,FEB,2.09,59.3,40.7
2,Alameda,1895.0,3.0,MAR,1.97,59.8,41.4
3,Alameda,1895.0,4.0,APR,1.75,65.2,43.8
4,Alameda,1895.0,5.0,MAY,1.20,71.2,47.5
...,...,...,...,...,...,...,...
90475,Yuba,2024.0,8.0,AUG,-9.99,-99.9,-99.9
90476,Yuba,2024.0,9.0,SEP,-9.99,-99.9,-99.9
90477,Yuba,2024.0,10.0,OCT,-9.99,-99.9,-99.9
90478,Yuba,2024.0,11.0,NOV,-9.99,-99.9,-99.9


In [1110]:
len(noaa_pivot_df["COUNTYNAME"].unique())

58

## Join Annual Census Data with Weather Data

In [1111]:
# Annual data only available from 2000 onward
housing_df = pd.read_csv("../Data/Housing Data/Processed Data/CA Housing 1960-2023.csv", index_col=0)
housing_df = housing_df[["CTYNAME"] + [f"{year}" for year in range(2000, 2021)]].copy()
housing_df.columns = ["COUNTYNAME"] + list(range(2000, 2021))
housing_df["COUNTYNAME"] = housing_df["COUNTYNAME"].str.split().apply(lambda row: ' '.join(row[:-1]))
housing_df.head(2)

,COUNTYNAME,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alameda,541385,546269,550054,553821,558677,564447,569206,575668,579064,...,584528,586639,589352,592396,595760,600467,605974,615030,622922,623368
1,Alpine,1519,1540,1589,1646,1670,1688,1701,1726,1755,...,1761,1764,1765,1764,1766,1769,1770,1778,1782,1543


In [886]:
housing_long = housing_df.melt(id_vars=['COUNTYNAME'], var_name='YEAR', value_name='Housing')
housing_long.head(2)

,COUNTYNAME,YEAR,Housing
0,Alameda,2000,541385
1,Alpine,2000,1519


In [1114]:
pop_df = pd.read_csv("../Data/Population Data/Processed Data/CA Population 1980-2023.csv", index_col=0)
len(pop_df["CTYNAME"].unique())

58

In [1115]:
pop_df = pd.read_csv("../CA Population 1980-2023.csv", index_col=0)
pop_df.columns.to_list()

['CTYNAME',
 'FULLFP',
 '1970',
 '1971',
 '1972',
 '1973',
 '1974',
 '1975',
 '1976',
 '1977',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 'Area_squaremiles',
 '1970 Population Density',
 '1971 Population Density',
 '1972 Population Density',
 '1973 Population Density',
 '1974 Population Density',
 '1975 Population Density',
 '1976 Population Density',
 '1977 Population Density',
 '1978 Population Density',
 '1979 Population Density',
 '1980 Population Density',
 '1981 Population Density',
 '1982 Population Density',
 '1983 Population Density',
 '1984 Population Density',
 '1985 Population Density',
 '1986 Population Dens

In [1116]:
# Data is available earlier than 2000 but limiting to 2000 to align with housing data availability
pop_density_df = pop_df[["CTYNAME"] + [f"{year} Population Density" for year in range(2000, 2021)]].copy()
pop_density_df.columns = ["COUNTYNAME"] + list(range(2000, 2021))
pop_density_df["COUNTYNAME"] = pop_density_df["COUNTYNAME"].str.split().apply(lambda row: ' '.join(row[:-1]))
pop_density_df.head(2)

,COUNTYNAME,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alameda,163.990499,166.212419,165.377898,164.479471,163.524601,163.052257,163.384685,164.655016,167.086076,...,173.158353,175.741319,178.657165,181.838932,184.857595,186.717000,187.741658,188.507635,188.713042,188.024319
1,Alpine,0.151125,0.149500,0.154625,0.154125,0.157875,0.152375,0.159500,0.159250,0.150250,...,0.136625,0.138875,0.141375,0.135375,0.135000,0.131625,0.139500,0.135125,0.139125,0.139875


In [1117]:
pop_density_long = pop_density_df.melt(id_vars=['COUNTYNAME'], var_name='YEAR', value_name='PopDensity')
pop_density_long.head(2)

,COUNTYNAME,YEAR,PopDensity
0,Alameda,2000,163.990499
1,Alpine,2000,0.151125


In [1118]:
housing_pop_df = pd.merge(pop_density_long, housing_long, on=['COUNTYNAME', 'YEAR'])
housing_pop_df["YEAR"] = housing_pop_df["YEAR"].astype(float)

noaa_housing_pop_df = pd.merge(noaa_pivot_df, housing_pop_df, on=['COUNTYNAME', 'YEAR'])
noaa_housing_pop_df

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing
0,Alameda,2000.0,1.0,JAN,5.94,57.0,42.8,163.990499,541385
1,Alameda,2000.0,2.0,FEB,7.24,57.7,44.3,163.990499,541385
2,Alameda,2000.0,3.0,MAR,1.92,63.1,43.8,163.990499,541385
3,Alameda,2000.0,4.0,APR,1.09,68.9,47.1,163.990499,541385
4,Alameda,2000.0,5.0,MAY,1.00,73.3,51.0,163.990499,541385
...,...,...,...,...,...,...,...,...,...
14611,Yuba,2020.0,8.0,AUG,0.05,95.8,65.0,11.567100,29566
14612,Yuba,2020.0,9.0,SEP,0.00,91.5,61.1,11.567100,29566
14613,Yuba,2020.0,10.0,OCT,0.00,85.4,53.3,11.567100,29566
14614,Yuba,2020.0,11.0,NOV,2.64,64.8,40.4,11.567100,29566


In [1120]:
# noaa_housing_pop_df.to_csv("../Data/noaa_housing_pop.csv")

## Merge DEM data

In [1123]:
dem_slope_df = pd.read_csv("../Data/dem_slope_data.csv", index_col=0)
dem_slope_df.head(2)

,slope_max,slope_mean,slope_min,slope_stdDev,CanonicalUrl
0,59.970421,29.091620,0.0,9.037646,/incidents/2013/8/17/rim-fire/
1,43.616798,18.365678,0.0,7.301831,/incidents/2013/5/30/powerhouse-fire/


In [1124]:
dem_elevation_df = pd.read_csv("../Data/dem_elevation_data.csv", index_col=0)
dem_elevation_df.head(2)

,elevation_max,elevation_mean,elevation_min,elevation_stdDev,CanonicalUrl
0,1233.0,721.992462,374.0,193.468704,/incidents/2013/8/17/rim-fire/
1,1336.0,902.059868,644.0,132.324578,/incidents/2013/5/30/powerhouse-fire/


In [1125]:
merged_dem_df = pd.merge(dem_slope_df, dem_elevation_df, 
                         on="CanonicalUrl", how="outer")
merged_dem_df.head(2)

,slope_max,slope_mean,slope_min,slope_stdDev,CanonicalUrl,elevation_max,elevation_mean,elevation_min,elevation_stdDev
0,59.970421,29.091620,0.0,9.037646,/incidents/2013/8/17/rim-fire/,1233.0,721.992462,374.0,193.468704
1,43.616798,18.365678,0.0,7.301831,/incidents/2013/5/30/powerhouse-fire/,1336.0,902.059868,644.0,132.324578


In [1126]:
dem_slope_df["CanonicalUrl"].value_counts()

CanonicalUrl
/incidents/2018/7/27/ranch-fire-mendocino-complex/             4
/incidents/2013/6/10/sonoma-lake-napa-unit-lightning-fires/    4
/incidents/2018/7/27/river-fire-mendocino-complex/             3
/incidents/2019/10/9/cross-fire/                               2
/incidents/2019/8/25/border-10-fire/                           2
                                                              ..
/incidents/2016/5/31/pala-fire/                                1
/incidents/2016/6/18/arroyo-fire/                              1
/incidents/2016/9/23/newmark-fire/                             1
/incidents/2016/9/10/lake-fire/                                1
/incidents/2019/10/14/johnson-fire/                            1
Name: count, Length: 1607, dtype: int64

## Load and filter wildfire data

In [1127]:
# from Kaggle: https://www.kaggle.com/datasets/ananthu017/california-wildfire-incidents-20132020

wildfire_fpath = "../Data/California_Fire_Incidents.csv"

In [1128]:
# Load wildfire data
wildfire_df = pd.read_csv(wildfire_fpath)
wildfire_df.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17T15:25:00Z,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30T15:28:00Z,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15T13:43:00Z,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10T16:30:00Z,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,...,"Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02T07:01:00Z,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [1129]:
# Retrieve year and month
wildfire_df["started_year"] = pd.to_datetime(wildfire_df["Started"], errors="coerce").dt.year
wildfire_df["started_month"] = pd.to_datetime(wildfire_df["Started"], errors="coerce").dt.month

# Handle conversion issues (4 rows)
problematic_rows = wildfire_df[wildfire_df['started_year'].isna()]
wildfire_df.loc[problematic_rows.index, 'started_year'] = problematic_rows['Started'].apply(lambda datetime_str: 
    datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ').year)
wildfire_df.loc[problematic_rows.index, 'started_month'] = problematic_rows['Started'].apply(lambda datetime_str:
    datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ').month)

In [1130]:
# Filter records missing lat/lon
missing_lonlat_df = wildfire_df[(wildfire_df["Longitude"] == 0.0) | (wildfire_df["Latitude"] == 0.0)]
wildfire_df_cleaned = wildfire_df[~wildfire_df["UniqueId"].isin(missing_lonlat_df["UniqueId"])]

In [1131]:
wildfire_df_cleaned["Counties"]

0          Tuolumne
1       Los Angeles
2         Riverside
3            Placer
5            Fresno
           ...     
1631      Riverside
1632         Nevada
1633           Yolo
1634      San Diego
1635      Riverside
Name: Counties, Length: 1482, dtype: object

In [1132]:
# Filter non-CA counties
ca_wildfire_df = wildfire_df_cleaned[~wildfire_df_cleaned["Counties"].isin(['Mexico', 'State of Oregon', 'State of Nevada'])]
ca_wildfire_df["Counties"].unique()

array(['Tuolumne', 'Los Angeles', 'Riverside', 'Placer', 'Fresno',
       'Siskiyou', 'Humboldt', 'Tehama', 'Shasta', 'San Diego', 'Kern',
       'Ventura', 'Sonoma', 'Contra Costa', 'Butte', 'Tulare',
       'Santa Barbara', 'Mariposa', 'El Dorado', 'San Bernardino',
       'Plumas', 'Modoc', 'San Luis Obispo', 'Madera', 'Inyo', 'Napa',
       'San Benito', 'San Joaquin', 'Alameda', 'Glenn', 'Yolo',
       'Sacramento', 'Stanislaus', 'Solano', 'Merced', 'Lake', 'Monterey',
       'Mendocino', 'Lassen', 'Amador', 'Yuba', 'Nevada', 'Santa Clara',
       'Calaveras', 'San Mateo', 'Orange', 'Colusa', 'Trinity', 'Alpine',
       'Mono', 'Sutter', 'Del Norte', 'Kings', 'Sierra', 'Santa Cruz',
       'Marin'], dtype=object)

In [1133]:
#### TODO: modify this if you want to export other columns
timeplace_colnames = ['started_year', 'started_month', 'Started', 'Extinguished', 'Location', 'Latitude', 'Longitude', 'Counties', 'Name']
target_colnames = ['AcresBurned', 'MajorIncident']

meta_colnames = ['UniqueId', 'CalFireIncident', 'CanonicalUrl']
impact_colnames = ['PercentContained', 'Fatalities', 'Injuries', 'StructuresDamaged', 'StructuresDestroyed', 'StructuresEvacuated', 'StructuresThreatened']
response_colnames = ['PersonnelInvolved', 'CrewsInvolved', 'WaterTenders', 'Helicopters', 'Dozers', 'Engines']
misc_colnames = ['ConditionStatement', 'FuelType', 'ControlStatement', 'Featured', 'Status', 'Final', 'Public', 'SearchDescription', 'SearchKeywords']

# Select specific columns
ca_wildfire_subset_df = ca_wildfire_df[meta_colnames + target_colnames + timeplace_colnames]
ca_wildfire_subset_df.head(2)

,UniqueId,CalFireIncident,CanonicalUrl,AcresBurned,MajorIncident,started_year,started_month,Started,Extinguished,Location,Latitude,Longitude,Counties,Name
0,5fb18d4d-213f-4d83-a179-daaf11939e78,True,/incidents/2013/8/17/rim-fire/,257314.0,False,2013.0,8.0,2013-08-17T15:25:00Z,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,37.857000,-120.086000,Tuolumne,Rim Fire
1,bf37805e-1cc2-4208-9972-753e47874c87,True,/incidents/2013/5/30/powerhouse-fire/,30274.0,False,2013.0,5.0,2013-05-30T15:28:00Z,2013-06-08T18:30:00Z,Angeles National Forest,34.585595,-118.423176,Los Angeles,Powerhouse Fire


In [1134]:
# Export cleaned wildfire data with subset of columns
ca_wildfire_subset_df.to_csv("../Data/ca_wildfire_cleaned.csv")

## Join cleaned wildfire data with environmental features

In [1135]:
ca_wildfire_subset_df.head(2)

,UniqueId,CalFireIncident,CanonicalUrl,AcresBurned,MajorIncident,started_year,started_month,Started,Extinguished,Location,Latitude,Longitude,Counties,Name
0,5fb18d4d-213f-4d83-a179-daaf11939e78,True,/incidents/2013/8/17/rim-fire/,257314.0,False,2013.0,8.0,2013-08-17T15:25:00Z,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,37.857000,-120.086000,Tuolumne,Rim Fire
1,bf37805e-1cc2-4208-9972-753e47874c87,True,/incidents/2013/5/30/powerhouse-fire/,30274.0,False,2013.0,5.0,2013-05-30T15:28:00Z,2013-06-08T18:30:00Z,Angeles National Forest,34.585595,-118.423176,Los Angeles,Powerhouse Fire


In [1136]:
noaa_housing_pop_df.head(2)

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing
0,Alameda,2000.0,1.0,JAN,5.94,57.0,42.8,163.990499,541385
1,Alameda,2000.0,2.0,FEB,7.24,57.7,44.3,163.990499,541385


In [1137]:
# Merge cleaned noaa_housing_pop data and wildfires data
merged_df_1 = pd.merge(noaa_housing_pop_df, ca_wildfire_subset_df, 
                     left_on=['COUNTYNAME', 'YEAR', 'MONTH_NUM'], 
                     right_on=['Counties', 'started_year', 'started_month'], how="right")
merged_df_1.head(2)

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing,UniqueId,...,MajorIncident,started_year,started_month,Started,Extinguished,Location,Latitude,Longitude,Counties,Name
0,Tuolumne,2013.0,8.0,AUG,0.10,80.1,51.3,2.204681,31332.0,5fb18d4d-213f-4d83-a179-daaf11939e78,...,False,2013.0,8.0,2013-08-17T15:25:00Z,2013-09-06T18:30:00Z,3 miles east of Groveland along Hwy 120,37.857000,-120.086000,Tuolumne,Rim Fire
1,Los Angeles,2013.0,5.0,MAY,0.55,78.1,53.6,195.294961,3473910.0,bf37805e-1cc2-4208-9972-753e47874c87,...,False,2013.0,5.0,2013-05-30T15:28:00Z,2013-06-08T18:30:00Z,Angeles National Forest,34.585595,-118.423176,Los Angeles,Powerhouse Fire


In [1160]:
# Load NDVI data
ndvi_summary_df = pd.read_csv("../Data/ndvi_summary_stats.csv")
ndvi_summary_df.head(2)

,UniqueId,average_ndvi,range_-1_0,range_0_0.2,range_0.2_0.5,range_0.5_1,percent_vegetated_below_0.5
0,79fd7b45-381a-4819-9dc7-f80a98010809,0.203354,0.121689,67.267227,31.092072,1.519012,98.479137
1,4affde91-31a9-4588-bf14-7201fc5c548b,0.184760,6.356049,65.796661,21.739960,6.107330,93.478137


In [1161]:
# Merge with NDVI data
merged_df_2 = pd.merge(merged_df_1, ndvi_summary_df, 
                       on="UniqueId", how="left")
merged_df_2.head(2)

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing,UniqueId,...,Latitude,Longitude,Counties,Name,average_ndvi,range_-1_0,range_0_0.2,range_0.2_0.5,range_0.5_1,percent_vegetated_below_0.5
0,Tuolumne,2013.0,8.0,AUG,0.10,80.1,51.3,2.204681,31332.0,5fb18d4d-213f-4d83-a179-daaf11939e78,...,37.857000,-120.086000,Tuolumne,Rim Fire,NaN,NaN,NaN,NaN,NaN,NaN
1,Los Angeles,2013.0,5.0,MAY,0.55,78.1,53.6,195.294961,3473910.0,bf37805e-1cc2-4208-9972-753e47874c87,...,34.585595,-118.423176,Los Angeles,Powerhouse Fire,NaN,NaN,NaN,NaN,NaN,NaN


In [1162]:
# Load ERA data
era_df = pd.read_csv("../Data/consolidated_wildfire_weather_data.csv")
era_df.head(2)

,UniqueId,latitude,longitude,closest_weather_lat,closest_weather_lon,year,date,season,season_year,24_hour_sp_min,...,year_t2m_mean,year_u10_min,year_u10_max,year_u10_mean,year_v10_min,year_v10_max,year_v10_mean,year_tp_min,year_tp_max,year_tp_mean
0,5fb18d4d-213f-4d83-a179-daaf11939e78,37.857000,-120.086000,37.75,-120.0,2013,2013-08-17,Summer,2013,90004.67850,...,286.664339,-4.364626,4.133791,-0.012735,-3.577216,5.170698,-0.123784,-0.001231,0.002853,0.000012
1,bf37805e-1cc2-4208-9972-753e47874c87,34.585595,-118.423176,34.50,-118.5,2013,2013-05-30,Spring,2013,92761.10596,...,289.997844,-7.206415,4.081632,0.186919,-6.491730,6.239663,-0.094266,-0.001486,0.001013,-0.000002


In [1163]:
era_df.columns

Index(['UniqueId', 'latitude', 'longitude', 'closest_weather_lat',
       'closest_weather_lon', 'year', 'date', 'season', 'season_year',
       '24_hour_sp_min', '24_hour_sp_max', '24_hour_sp_mean',
       '24_hour_d2m_min', '24_hour_d2m_max', '24_hour_d2m_mean',
       '24_hour_t2m_min', '24_hour_t2m_max', '24_hour_t2m_mean',
       '24_hour_u10_min', '24_hour_u10_max', '24_hour_u10_mean',
       '24_hour_v10_min', '24_hour_v10_max', '24_hour_v10_mean',
       '24_hour_tp_min', '24_hour_tp_max', '24_hour_tp_mean', 'season_sp_min',
       'season_sp_max', 'season_sp_mean', 'season_d2m_min', 'season_d2m_max',
       'season_d2m_mean', 'season_t2m_min', 'season_t2m_max',
       'season_t2m_mean', 'season_u10_min', 'season_u10_max',
       'season_u10_mean', 'season_v10_min', 'season_v10_max',
       'season_v10_mean', 'season_tp_min', 'season_tp_max', 'season_tp_mean',
       'year_sp_min', 'year_sp_max', 'year_sp_mean', 'year_d2m_min',
       'year_d2m_max', 'year_d2m_mean', 'year_t2m_

In [1164]:
# Merge ERA data
merged_df_3 = pd.merge(merged_df_2, era_df, 
                       on="UniqueId", how="left")
merged_df_3.head(2)

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing,UniqueId,...,year_t2m_mean,year_u10_min,year_u10_max,year_u10_mean,year_v10_min,year_v10_max,year_v10_mean,year_tp_min,year_tp_max,year_tp_mean
0,Tuolumne,2013.0,8.0,AUG,0.10,80.1,51.3,2.204681,31332.0,5fb18d4d-213f-4d83-a179-daaf11939e78,...,286.664339,-4.364626,4.133791,-0.012735,-3.577216,5.170698,-0.123784,-0.001231,0.002853,0.000012
1,Los Angeles,2013.0,5.0,MAY,0.55,78.1,53.6,195.294961,3473910.0,bf37805e-1cc2-4208-9972-753e47874c87,...,289.997844,-7.206415,4.081632,0.186919,-6.491730,6.239663,-0.094266,-0.001486,0.001013,-0.000002


In [1165]:
merged_df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 91 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COUNTYNAME                   1476 non-null   object 
 1   YEAR                         1476 non-null   float64
 2   MONTH_NUM                    1476 non-null   float64
 3   MONTH                        1476 non-null   object 
 4   pcp                          1476 non-null   float64
 5   tmax                         1476 non-null   float64
 6   tmin                         1476 non-null   float64
 7   PopDensity                   1476 non-null   float64
 8   Housing                      1476 non-null   float64
 9   UniqueId                     1478 non-null   object 
 10  CalFireIncident              1478 non-null   bool   
 11  CanonicalUrl                 1478 non-null   object 
 12  AcresBurned                  1475 non-null   float64
 13  MajorIncident     

In [1166]:
# Export data
merged_df_3[[
       'COUNTYNAME', 'Started', 'Extinguished', 'Location', 'Latitude',
       'Longitude', 'Name',  'AcresBurned', 'MajorIncident', 'PopDensity', 'Housing',
       # NOAA Data
       'pcp', 'tmax', 'tmin',
       # ERA Data
       'closest_weather_lat', 'closest_weather_lon', 'season', 'season_year',
       '24_hour_sp_min', '24_hour_sp_max', '24_hour_sp_mean',
       '24_hour_d2m_min', '24_hour_d2m_max', '24_hour_d2m_mean',
       '24_hour_t2m_min', '24_hour_t2m_max', '24_hour_t2m_mean',
       '24_hour_u10_min', '24_hour_u10_max', '24_hour_u10_mean',
       '24_hour_v10_min', '24_hour_v10_max', '24_hour_v10_mean',
       '24_hour_tp_min', '24_hour_tp_max', '24_hour_tp_mean', 'season_sp_min',
       'season_sp_max', 'season_sp_mean', 'season_d2m_min', 'season_d2m_max',
       'season_d2m_mean', 'season_t2m_min', 'season_t2m_max',
       'season_t2m_mean', 'season_u10_min', 'season_u10_max',
       'season_u10_mean', 'season_v10_min', 'season_v10_max',
       'season_v10_mean', 'season_tp_min', 'season_tp_max', 'season_tp_mean',
       'year_sp_min', 'year_sp_max', 'year_sp_mean', 'year_d2m_min',
       'year_d2m_max', 'year_d2m_mean', 'year_t2m_min', 'year_t2m_max',
       'year_t2m_mean', 'year_u10_min', 'year_u10_max', 'year_u10_mean',
       'year_v10_min', 'year_v10_max', 'year_v10_mean', 'year_tp_min',
       'year_tp_max', 'year_tp_mean',
       # Vegetation Data
       'average_ndvi', 'percent_vegetated_below_0.5', 
       'range_-1_0', 'range_0_0.2', 'range_0.2_0.5', 'range_0.5_1', 
       # Meta Data
       'YEAR', 'MONTH_NUM', 'UniqueId', 'CalFireIncident', 'CanonicalUrl'
]].sort_values(by="Started", ascending=False)\
.to_csv("../Data/wildfires_env_072924_no_elevation.csv")

In [1167]:
merged_no_elev_df = pd.read_csv("../Data/wildfires_env_072924_no_elevation.csv", index_col=0)
merged_no_elev_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1478 entries, 1257 to 872
Data columns (total 83 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COUNTYNAME                   1476 non-null   object 
 1   Started                      1478 non-null   object 
 2   Extinguished                 1419 non-null   object 
 3   Location                     1478 non-null   object 
 4   Latitude                     1478 non-null   float64
 5   Longitude                    1478 non-null   float64
 6   Name                         1478 non-null   object 
 7   AcresBurned                  1475 non-null   float64
 8   MajorIncident                1478 non-null   bool   
 9   PopDensity                   1476 non-null   float64
 10  Housing                      1476 non-null   float64
 11  pcp                          1476 non-null   float64
 12  tmax                         1476 non-null   float64
 13  tmin                 

In [1168]:
sum(~merged_no_elev_df["average_ndvi"].isna())

1141

In [1169]:
#### MERGE AND EXPORT AFTER DEM ELEVATION DATA IS FIXED ####

In [1170]:
# Merge with DEM data
merged_df_4 = pd.merge(merged_df_3, merged_dem_df, 
                       on="CanonicalUrl", how="left")
merged_df_4.head(2)

,COUNTYNAME,YEAR,MONTH_NUM,MONTH,pcp,tmax,tmin,PopDensity,Housing,UniqueId,...,year_tp_max,year_tp_mean,slope_max,slope_mean,slope_min,slope_stdDev,elevation_max,elevation_mean,elevation_min,elevation_stdDev
0,Tuolumne,2013.0,8.0,AUG,0.10,80.1,51.3,2.204681,31332.0,5fb18d4d-213f-4d83-a179-daaf11939e78,...,0.002853,0.000012,59.970421,29.091620,0.0,9.037646,1233.0,721.992462,374.0,193.468704
1,Los Angeles,2013.0,5.0,MAY,0.55,78.1,53.6,195.294961,3473910.0,bf37805e-1cc2-4208-9972-753e47874c87,...,0.001013,-0.000002,43.616798,18.365678,0.0,7.301831,1336.0,902.059868,644.0,132.324578


In [1171]:
merged_df_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 99 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COUNTYNAME                   1731 non-null   object 
 1   YEAR                         1731 non-null   float64
 2   MONTH_NUM                    1731 non-null   float64
 3   MONTH                        1731 non-null   object 
 4   pcp                          1731 non-null   float64
 5   tmax                         1731 non-null   float64
 6   tmin                         1731 non-null   float64
 7   PopDensity                   1731 non-null   float64
 8   Housing                      1731 non-null   float64
 9   UniqueId                     1733 non-null   object 
 10  CalFireIncident              1733 non-null   bool   
 11  CanonicalUrl                 1733 non-null   object 
 12  AcresBurned                  1730 non-null   float64
 13  MajorIncident     

In [1172]:
# Export data
merged_df_4[[
       'COUNTYNAME', 'Started', 'Extinguished', 'Location', 'Latitude',
       'Longitude', 'Name',  'AcresBurned', 'MajorIncident', 'PopDensity', 'Housing',
       # NOAA Data
       'pcp', 'tmax', 'tmin',
       # ERA Data
       'closest_weather_lat', 'closest_weather_lon', 'season', 'season_year',
       '24_hour_sp_min', '24_hour_sp_max', '24_hour_sp_mean',
       '24_hour_d2m_min', '24_hour_d2m_max', '24_hour_d2m_mean',
       '24_hour_t2m_min', '24_hour_t2m_max', '24_hour_t2m_mean',
       '24_hour_u10_min', '24_hour_u10_max', '24_hour_u10_mean',
       '24_hour_v10_min', '24_hour_v10_max', '24_hour_v10_mean',
       '24_hour_tp_min', '24_hour_tp_max', '24_hour_tp_mean', 'season_sp_min',
       'season_sp_max', 'season_sp_mean', 'season_d2m_min', 'season_d2m_max',
       'season_d2m_mean', 'season_t2m_min', 'season_t2m_max',
       'season_t2m_mean', 'season_u10_min', 'season_u10_max',
       'season_u10_mean', 'season_v10_min', 'season_v10_max',
       'season_v10_mean', 'season_tp_min', 'season_tp_max', 'season_tp_mean',
       'year_sp_min', 'year_sp_max', 'year_sp_mean', 'year_d2m_min',
       'year_d2m_max', 'year_d2m_mean', 'year_t2m_min', 'year_t2m_max',
       'year_t2m_mean', 'year_u10_min', 'year_u10_max', 'year_u10_mean',
       'year_v10_min', 'year_v10_max', 'year_v10_mean', 'year_tp_min',
       'year_tp_max', 'year_tp_mean',
       # DEM Data
       'slope_max', 'slope_mean', 'slope_min', 'slope_stdDev', 
       'elevation_max', 'elevation_mean', 'elevation_min', 'elevation_stdDev',
       # Vegetation Data
       'average_ndvi', 'percent_vegetated_below_0.5', 
       'range_-1_0', 'range_0_0.2', 'range_0.2_0.5', 'range_0.5_1', 
       # Meta Data
       'YEAR', 'MONTH_NUM', 'UniqueId', 'CalFireIncident', 'CanonicalUrl'
]].sort_values(by="Started", ascending=False)\
.to_csv("../Data/wildfires_env_072924.csv")

In [1173]:
cleaned_merged_df = pd.read_csv("../Data/wildfires_env_072924.csv", index_col=0)
cleaned_merged_df

,COUNTYNAME,Started,Extinguished,Location,Latitude,Longitude,Name,AcresBurned,MajorIncident,PopDensity,...,percent_vegetated_below_0.5,range_-1_0,range_0_0.2,range_0.2_0.5,range_0.5_1,YEAR,MONTH_NUM,UniqueId,CalFireIncident,CanonicalUrl
1509,Santa Barbara,2019-11-25T19:59:12Z,2019-12-14T08:22:00Z,Highway 154 and E Camino Cielo near Painted Ca...,34.502500,-119.785000,Cave Fire (No Longer a CAL FIRE Incident),3126.0,False,10.922251,...,55.865781,0.001907,3.038406,52.826309,44.133377,2019.0,11.0,7555692b-f672-4660-8f15-d0dab068a193,False,/incidents/2019/11/25/cave-fire-no-longer-a-ca...
1468,Placer,2019-11-25T12:39:44Z,NaN,"Foothills and McAnally Drive, northwest of Ros...",38.838992,-121.325842,Foothills Fire,355.0,False,24.667203,...,99.933845,2.549362,92.014694,5.371475,0.064468,2019.0,11.0,77d64e4c-a50f-4f4d-a750-e6a6f0ddadc9,True,/incidents/2019/11/25/foothills-fire/
1730,Yolo,2019-11-25T12:02:02Z,NaN,"County Road 102 and County Road 17, North of W...",38.734634,-121.729691,Cashe Fire,NaN,False,19.970863,...,94.224848,0.510025,76.155853,17.588425,5.745697,2019.0,11.0,9c26f915-1b33-422d-b30a-9eb4da6fd729,False,/incidents/2019/11/25/cashe-fire/
1628,Lake,2019-11-05T06:52:14Z,2019-11-06T18:30:00Z,"Walker Ridge Road and Bartlett Springs Road, n...",39.100408,-122.496548,Eagle Fire,75.0,False,4.504752,...,75.755692,0.000000,5.380249,70.375443,24.244308,2019.0,11.0,600d608d-6279-435d-bc9b-99aacaa5e80f,True,/incidents/2019/11/5/eagle-fire/
1511,Tehama,2019-11-03T14:16:17Z,2019-11-14T18:02:00Z,"Colyer Springs Rd and Raglin Ridge Rd, southwe...",40.036379,-122.637837,Ranch Fire,2534.0,True,2.044474,...,87.695478,0.035858,4.796982,82.867050,12.300110,2019.0,11.0,69d98a94-4980-4dc0-b388-61c37c635f32,True,/incidents/2019/11/3/ranch-fire/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,Madera,2013-04-30T12:59:00Z,2013-05-01T07:00:00Z,Between Road 210 and Road 200 near Fine Gold C...,37.116295,-119.635004,Gold Fire,274.0,False,6.527526,...,NaN,NaN,NaN,NaN,NaN,2013.0,4.0,357ffc13-bef9-48eb-810f-c5de851972eb,True,/incidents/2013/4/30/gold-fire/
143,San Bernardino,2013-04-20T17:30:00Z,2013-04-22T09:00:00Z,"west of Delamar Mountain, north of the communi...",34.288877,-116.941311,Fawnskin Fire,30.0,False,9.621094,...,NaN,NaN,NaN,NaN,NaN,2013.0,4.0,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,False,/incidents/2013/4/20/fawnskin-fire/
45,Inyo,2013-02-24T08:16:00Z,2013-02-28T20:00:00Z,"south of Narrow Gauge Rd & north of Hwy 136, e...",36.602575,-118.016510,River Fire,406.0,True,0.166677,...,NaN,NaN,NaN,NaN,NaN,2013.0,2.0,094719ba-a47b-4abb-9ec5-a506b2b9fd23,True,/incidents/2013/2/24/river-fire/
1010,NaN,1969-12-31T16:00:00Z,2018-01-09T10:05:00Z,"Interstate 5 near Taglio Road, Gustine",37.218120,-121.077610,Taglio Fire,12.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fcb8579e-7469-4c9e-9eac-527264e6f26b,True,/incidents/2017/12/31/taglio-fire/
